# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
cities_df = pd.read_csv('output_data/cities.csv') # using temp file. Use actual file that was saved in weatherpy
cities_df.head()

,Unnamed: 0,City,country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,saint-pierre,RE,-21.3393,55.4781,77.0,73,90,21.85,1619490028
1,1,cape town,ZA,-33.9258,18.4232,63.0,94,90,3.44,1619489889
2,2,punta arenas,CL,-53.1500,-70.9167,42.8,100,90,3.44,1619489651
3,3,hermanus,ZA,-34.4187,19.2345,61.0,88,98,2.66,1619489890
4,4,qaanaaq,GL,77.4840,-69.3632,35.6,69,100,7.31,1619490404


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
locations = cities_df[["Lat", "Lon"]]
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()
max_humidity

100.0

In [36]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
perfect_weather_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
perfect_weather_df.dropna(how='any')
perfect_weather_df


,Unnamed: 0,City,country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
77,77,inhambane,MZ,-23.8650,35.3833,71.08,89,0,4.79,1619490421
112,112,carnarvon,AU,-24.8667,113.6333,78.80,47,0,3.44,1619489912
160,160,porto nacional,BR,-10.7081,-48.4172,75.20,100,0,3.44,1619490443
175,175,riyadh,SA,24.6877,46.7219,75.20,22,0,9.31,1619489628
202,202,esmeraldas,BR,-19.7625,-44.3139,73.40,78,0,2.30,1619490453
238,238,alta floresta,BR,-9.8756,-56.0861,73.40,100,0,2.30,1619489936
343,343,yaypan,UZ,40.3758,70.8156,71.53,29,0,4.72,1619490492
400,400,santa cruz cabralia,BR,-16.2781,-39.0247,75.20,78,0,5.75,1619490508
517,517,parrita,CR,9.5201,-84.3225,71.60,78,0,9.22,1619490545


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# set up additional columns to hold information
hotel_df = perfect_weather_df
hotel_df['Hotel'] = ""
hotel_df

<ipython-input-42-57f0f40a7529>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel'] = ""


,Unnamed: 0,City,country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel
77,77,inhambane,MZ,-23.8650,35.3833,71.08,89,0,4.79,1619490421,
112,112,carnarvon,AU,-24.8667,113.6333,78.80,47,0,3.44,1619489912,
160,160,porto nacional,BR,-10.7081,-48.4172,75.20,100,0,3.44,1619490443,
175,175,riyadh,SA,24.6877,46.7219,75.20,22,0,9.31,1619489628,
202,202,esmeraldas,BR,-19.7625,-44.3139,73.40,78,0,2.30,1619490453,
238,238,alta floresta,BR,-9.8756,-56.0861,73.40,100,0,2.30,1619489936,
343,343,yaypan,UZ,40.3758,70.8156,71.53,29,0,4.72,1619490492,
400,400,santa cruz cabralia,BR,-16.2781,-39.0247,75.20,78,0,5.75,1619490508,
517,517,parrita,CR,9.5201,-84.3225,71.60,78,0,9.22,1619490545,


In [43]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lon = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/kinjalmaru/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
Missing field/result... skipping.


In [44]:
hotel_df

,Unnamed: 0,City,country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel
77,77,inhambane,MZ,-23.8650,35.3833,71.08,89,0,4.79,1619490421,Sentidos Beach Retreat
112,112,carnarvon,AU,-24.8667,113.6333,78.80,47,0,3.44,1619489912,Hospitality Carnarvon
160,160,porto nacional,BR,-10.7081,-48.4172,75.20,100,0,3.44,1619490443,Vila Real Hotel
175,175,riyadh,SA,24.6877,46.7219,75.20,22,0,9.31,1619489628,Hyatt Regency Riyadh Olaya
202,202,esmeraldas,BR,-19.7625,-44.3139,73.40,78,0,2.30,1619490453,Parque do Avestruz Hotel Fazenda
238,238,alta floresta,BR,-9.8756,-56.0861,73.40,100,0,2.30,1619489936,Hotel Caleche Park Hotel
343,343,yaypan,UZ,40.3758,70.8156,71.53,29,0,4.72,1619490492,
400,400,santa cruz cabralia,BR,-16.2781,-39.0247,75.20,78,0,5.75,1619490508,Campo Bahia
517,517,parrita,CR,9.5201,-84.3225,71.60,78,0,9.22,1619490545,Hotel Cocomar


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dl>
<dt>Name</dt><dd>{Hotel}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

KeyError: 'Country'

In [32]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…